### **1. Mengimpor Library**

In [57]:
!pip install pyswarm

  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4464 sha256=28f65c3c1b431a68fc38037b1e42298084907afe0a0f11a065bc228204186f50
  Stored in directory: /root/.cache/pip/wheels/71/67/40/62fa158f497f942277cbab8199b05cb61c571ab324e67ad0d6
Successfully built pyswarm


In [58]:
#---MULAI---
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from xgboost import XGBRegressor
from pyswarm import pso
#---SELESAI---

### **2. Mengambil Data dari Yahoo Finance**

In [59]:
#---MULAI---
# Ticker saham
ticker = "KLBF.JK"

# Rentang waktu
start_date = "2019-01-01"
end_date = "2024-12-31"

# Ambil data historis menggunakan yfinance
data = yf.download(ticker, start=start_date, end=end_date)

# Simpan data ke file CSV
data.to_csv("KLBF_JK_Historical.csv")

# Tampilkan 5 baris pertama
print(data.head())
#---SELESAI---

[*********************100%***********************]  1 of 1 completed

Price         Adj Close   Close    High     Low    Open    Volume
Ticker          KLBF.JK KLBF.JK KLBF.JK KLBF.JK KLBF.JK   KLBF.JK
Date                                                             
2019-01-01  1353.745850  1520.0  1520.0  1520.0  1520.0         0
2019-01-02  1358.198975  1525.0  1530.0  1500.0  1525.0   5035800
2019-01-03  1371.558350  1540.0  1545.0  1505.0  1535.0  15603900
2019-01-04  1398.276978  1570.0  1575.0  1520.0  1535.0  19765600
2019-01-07  1420.542480  1595.0  1610.0  1580.0  1580.0  28281300


### **3. Persiapan Dataset**

In [60]:
#---MULAI---
# Tambahkan target Next_Day_Close
data['Next_Day_Close'] = data['Close'].shift(-1)
data.dropna(inplace=True)

# Pilih fitur dan target
features = ['Close']
target = 'Next_Day_Close'

X = data[features]
y = data[target]
#---SELESAI---

### **4. Normalisasi Data**

In [61]:
#---MULAI---
# Normalisasi fitur
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
#---SELESAI---

### **5. Membagi Data Latih dan Uji**

In [68]:
#---MULAI---
# Bagi data menjadi data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Data Training dan Testing untuk Skenario 2 (XGBoost-PSO)
x_train2 = X_train.copy()  # Salin data latih untuk skenario 2
y_train2 = y_train.copy()
x_test2 = X_test.copy()    # Salin data uji untuk skenario 2
y_test2 = y_test.copy()
#---SELESAI---

### **6. Melatih Model XGBoost**




In [70]:
#---MULAI---
# Train model XGBoost
model = xgb.XGBRegressor(random_state=42)
model.fit(X_train, y_train)

# Simpan model ke file
joblib.dump(model, "xgboost_model_klbf.pkl")
#---SELESAI---

['xgboost_model_klbf.pkl']

### **7. Eksperimen Parameter XGBoost**

#### **7.1 XGBoost tanpa optimasi PSO**



In [73]:
# --- MULAI ---
# Training Model XGBoost dengan Parameter default
model_default = XGBRegressor(
    max_depth=6,
    gamma=0,
    reg_lambda=1,
    learning_rate=0.3,
    min_child_weight=1,
    subsample=1,
    colsample_bytree=1
)

model_default.fit(X_train, y_train)

# Simpan model default ke file
joblib.dump(model_default, "xgboost_model_default.pkl")
# --- SELESAI ---

['xgboost_model_default.pkl']

#### **7.2 XGBoost dengan optimasi parameter PSO**

In [78]:
# --- MULAI ---
# Training Model XGBoost dengan optimasi parameter PSO
def tune_xgboost(params):
    max_depth = int(params[0])
    gamma = params[1]
    reg_lambda = params[2]
    learning_rate = params[3]
    min_child_weight = params[4]
    subsample = params[5]
    colsample_bytree = params[6]

    model_pso = XGBRegressor(
        max_depth=max_depth,
        gamma=gamma,
        reg_lambda=reg_lambda,
        learning_rate=learning_rate,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree
    )

    model_pso.fit(x_train2, y_train2)
    pred2 = model_pso.predict(x_test2)
    mse2 = mean_squared_error(y_test2, pred2)

    return mse2

# Optimasi Parameter dengan PSO
lb = [3, 0.01, 0.01, 0.01, 1, 0.5, 0.6]  # Lower Bound
ub = [10, 0.5, 10, 0.3, 10, 0.9, 1]      # Upper Bound

start_time = time.time()

best_params, _ = pso(
    tune_xgboost,
    lb,
    ub,
    swarmsize=100,
    omega=0.6,
    phip=1.5,
    phig=1.5,
    maxiter=100,
    minfunc=1e-12,
    debug=True
)

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nAlgoritma PSO Membutuhkan Waktu {elapsed_time} Detik")

# Best Parameter Hasil PSO
best_max_depth = int(best_params[0])
best_gamma = best_params[1]
best_reg_lambda = best_params[2]
best_learning_rate = best_params[3]
best_min_child_weight = best_params[4]
best_subsample = best_params[5]
best_colsample_bytree = best_params[6]

print('Best max_depth :', best_max_depth)
print('Best gamma :', best_gamma)
print('Best reg_lambda :', best_reg_lambda)
print('Best learning_rate :', best_learning_rate)
print('Best min_child_weight :', best_min_child_weight)
print('Best subsample :', best_subsample)
print('Best colsample_bytree :', best_colsample_bytree)

# Latih ulang model dengan parameter terbaik
model_pso = XGBRegressor(
    max_depth=best_max_depth,
    gamma=best_gamma,
    reg_lambda=best_reg_lambda,
    learning_rate=best_learning_rate,
    min_child_weight=best_min_child_weight,
    subsample=best_subsample,
    colsample_bytree=best_colsample_bytree
)

model_pso.fit(x_train2, y_train2)

# Simpan model PSO ke file setelah eksperimen
joblib.dump(model_pso, "xgboost_model_pso.pkl")
print("Model hasil optimasi PSO telah disimpan ke file 'xgboost_model_pso.pkl'")
# --- SELESAI ---

No constraints given.
New best for swarm at iteration 1: [3.         0.5        4.03528943 0.17938565 5.63835559 0.76131467
 0.74925144] 975.2607071276273
Best after iteration 1: [3.         0.5        4.03528943 0.17938565 5.63835559 0.76131467
 0.74925144] 975.2607071276273
New best for swarm at iteration 2: [ 3.          0.5        10.          0.17068648  7.54830102  0.71361746
  0.6777502 ] 975.2054248584852
Best after iteration 2: [ 3.          0.5        10.          0.17068648  7.54830102  0.71361746
  0.6777502 ] 975.2054248584852
New best for swarm at iteration 3: [3.         0.5        3.41016308 0.13316224 5.17368702 0.8707542
 0.69170206] 973.5773520143766
New best for swarm at iteration 3: [3.         0.46196467 4.33602708 0.08426528 5.75951213 0.85379348
 0.61864825] 973.2636335963244
Best after iteration 3: [3.         0.46196467 4.33602708 0.08426528 5.75951213 0.85379348
 0.61864825] 973.2636335963244
New best for swarm at iteration 4: [3.         0.5        4.1288655

In [80]:
# --- MULAI ---
# Training Best Model XGBoost dengan Parameter Terbaik hasil PSO
best_model = XGBRegressor(
    max_depth=3,  # Hasil dari PSO
    gamma=0.4998,  # Hasil dari PSO
    reg_lambda=2.8687,  # Hasil dari PSO
    learning_rate=0.1643,  # Hasil dari PSO
    min_child_weight=7.7663,  # Hasil dari PSO
    subsample=0.8618,  # Hasil dari PSO
    colsample_bytree=0.6009,  # Hasil dari PSO
    random_state=42  # Untuk reproduktifitas
)

# Fit model ke data latih
best_model.fit(x_train2, y_train2)

# Simpan best model ke file
joblib.dump(best_model, "best_xgboost_model.pkl")

print("Best model berhasil dilatih dan disimpan sebagai 'best_xgboost_model.pkl'")
# --- SELESAI ---

Best model berhasil dilatih dan disimpan sebagai 'best_xgboost_model.pkl'


### **8. Prediksi dan Evaluasi Hasil**

In [81]:
#---MULAI---
# Langkah 8: Prediksi dan Evaluasi

# Prediksi data uji untuk Model 1 (XGBoost tanpa optimasi PSO)
y_pred = model.predict(X_test)

# Prediksi data uji untuk Model 2 (XGBoost dengan optimasi parameter dari PSO)
y_pred2 = best_model.predict(x_test2)

# Pastikan y_test dan y_test2 adalah numerik
if y_test.dtype == 'object':
    y_test = y_test.astype(float)

if y_test2.dtype == 'object':
    y_test2 = y_test2.astype(float)

# Evaluasi Hasil untuk Model 1
mse1 = mean_squared_error(y_test, y_pred)
rmse1 = np.sqrt(mse1)
mae1 = mean_absolute_error(y_test, y_pred)
mape1 = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
r2_1 = r2_score(y_test, y_pred)

print("=== Model 1: XGBoost tanpa optimasi PSO ===")
print("Mean Squared Error (MSE):", mse1)
print("Root Mean Squared Error (RMSE):", rmse1)
print("Mean Absolute Error (MAE):", mae1)
print("Mean Absolute Percentage Error (MAPE):", mape1)
print("R-squared:", r2_1)

# Evaluasi Hasil untuk Model 2
mse2 = mean_squared_error(y_test2, y_pred2)
rmse2 = np.sqrt(mse2)
mae2 = mean_absolute_error(y_test2, y_pred2)
mape2 = np.mean(np.abs((y_test2 - y_pred2) / y_test2)) * 100
r2_2 = r2_score(y_test2, y_pred2)

print("\n=== Model 2: XGBoost dengan optimasi PSO ===")
print("Mean Squared Error (MSE):", mse2)
print("Root Mean Squared Error (RMSE):", rmse2)
print("Mean Absolute Error (MAE):", mae2)
print("Mean Absolute Percentage Error (MAPE):", mape2)
print("R-squared:", r2_2)

# Save Model
pickle.dump(model, open('model_xgboost.pkl', 'wb'))
pickle.dump(best_model, open('model_xgboost_pso.pkl', 'wb'))

print("\nModel 1 dan Model 2 berhasil disimpan ke file.")
#---SELESAI---

=== Model 1: XGBoost tanpa optimasi PSO ===
Mean Squared Error (MSE): 1302.852330038552
Root Mean Squared Error (RMSE): 36.09504578246926
Mean Absolute Error (MAE): 25.611528046277105
Mean Absolute Percentage Error (MAPE): 1.6174815854351117
R-squared: 0.974775906029878

=== Model 2: XGBoost dengan optimasi PSO ===
Mean Squared Error (MSE): 1004.3639280906036
Root Mean Squared Error (RMSE): 31.69170124954802
Mean Absolute Error (MAE): 23.90872337704613
Mean Absolute Percentage Error (MAPE): 1.4901853443630326
R-squared: 0.9805548414672531

Model 1 dan Model 2 berhasil disimpan ke file.


### **9. GUI**

#### **9.1 Backend Flask**

In [84]:
from flask import Flask, render_template, request
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import numpy as np

# Load model
model = pickle.load(open("model_xgboost_pso.pkl", "rb"))

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=["POST"])
def predict():
    close_price = float(request.form['close'])

    # Predict next day close price
    input_data = [[close_price]]
    predicted_close = model.predict(input_data)[0]

    # Example evaluation metrics (static for demo)
    y_test = np.array([1500, 1520, 1535, 1550])
    y_pred = np.array([1495, 1525, 1530, 1545])
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    metrics = {
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'r2': r2
    }

    return render_template(
        'index.html',
        prediction=f"Harga prediksi untuk hari berikutnya: {predicted_close:.2f}",
        metrics=metrics
    )

if __name__ == "__main__":
    app.run(debug=True, port=5000)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
